In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from pandas import Series, DataFrame
import pandas as pd
import numpy as np

from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from tqdm import trange
import time
import random
import json

In [ ]:
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
chrome_options = webdriver.chrome.options.Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument('--start-maximized')  # 전체 화면으로 시작
driver = webdriver.Chrome(options=chrome_options)

## 예시

# 리스트 파악

https://www.wanted.co.kr/wdlist/518?country=kr&job_sort=job.recommend_order&years=-1&locations=all
여기 특정 검색 및 태그로 검색 한 후 전체 리스트를 스크롤 end 통해 계속 내려끝까지 확인 

In [ ]:
element_xpath = "//div[@data-cy=\"job-card\"]/a"  
list_df = pd.DataFrame(columns=['job_category_id','job_category', 'company_id','company_name','position_name','position_id','link'])
list_num = len(driver.find_elements(By.XPATH, element_xpath))
list_num

52

In [ ]:

for i in trange(list_num):
    driver_i = driver.find_elements(By.XPATH, element_xpath)[i]
    job_category_id = driver_i.get_attribute('data-job-category-id')
    job_category = driver_i.get_attribute('data-job-category')
    company_id = driver_i.get_attribute('data-company-id')
    company_name = driver_i.get_attribute('data-company-name')
    position_name = driver_i.get_attribute('data-position-name')
    position_id = driver_i.get_attribute('data-position-id')
    href  = driver_i.get_attribute('href')

    list_df2 = pd.DataFrame({'job_category_id':pd.Series(job_category_id),'job_category':pd.Series(job_category),
                             'company_id':pd.Series(company_id),'company_name':pd.Series(company_name),'position_name':pd.Series(position_name),
                             'position_id':pd.Series(position_id),'link':pd.Series(href)})
    list_df = pd.concat([list_df, list_df2], ignore_index=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 52/52 [00:02<00:00, 22.80it/s]


In [ ]:
list_df.to_excel('c://python//원티드_연봉10%.xlsx', index=False)

※주의 특정 검색 및 태그로 설정한 후 노출 된 것들 중심으로 확인후 리스트 뽑아서 엑셀로 vlookup 작업하여(공고 ID값을 기본키로) 아래 칼럼 추가할것 

1. 전체 대상 2. 연봉 10% 상위(top_salary 칼럼 추가 후 Y ) 3. 대기업 임직원 1000명 이상(major_company 칼럼 추가 후 Y )

In [120]:

list_df['company'] = '-'
list_df['position'] = '-'
list_df['content1'] = '-'
list_df['content2'] = '-'
list_df['content3'] = '-'
list_df['content4'] = '-'
list_df['skill'] = '-'
list_df['period'] = '-'


## 리스트 작업 중간에 불러올 경우

In [8]:
##list_df = pd.read_excel('c://python//원티드_241203_개발자리스트.xlsx') 
list_df = pd.read_excel('c://python//원티드_크롤링_IT직군대상.xlsx') 

In [6]:
list_index = np.where( (list_df['top_salary']=='Y') | (list_df['major_company']=='Y'))[0]

In [7]:
len(list_index)

1489

In [56]:
for i in list_index[np.where( list_index == 3120)[0][0]:]  :  #최초값은  list_index 
    driver.get(list_df['link'][i])
    time.sleep(3)  # 페이지 로드 대기
    src = driver.page_source
    bs = BeautifulSoup(src, 'lxml')
    description_meta = bs.find("meta", {"name": "description"}).get("content")

    # META 태그 정보 추출
    time.sleep(1)
    element_xpath = "//span[contains(text(),'상세 정보 더 보기')]/parent::button" 
    driver.find_element(By.XPATH, element_xpath).click()
        
    element_xpath0 =  "//div[contains(@class, 'JobHeader_JobHeader__Tools')]"
    list_df['company'][i] = driver.find_element(By.XPATH, element_xpath0).text
    list_df['position'][i]  = position = description_meta.split("포지션: ")[1].split("\n")[0].strip()

    list_df['content1'][i]  = "".join(description_meta.split("주요 업무: ")[1].split("\n")).replace('• ','').replace('', '') #주요업무

    element_xpath_content2 = "//h3[contains(text(),'자격요건')]/parent::div" 
    list_df['content2'][i]  = driver.find_element(By.XPATH,element_xpath_content2).text.replace('자격요건\n','').replace('• ','').replace('', '') #자격요건

    try:
        element_xpath_content3 = "//h3[contains(text(),'우대사항')]/parent::div" 
        list_df['content3'][i]  = driver.find_element(By.XPATH,element_xpath_content3).text.replace('우대사항\n','').replace('• ','').replace('', '') #우대사항
    except:
        list_df['content3'][i]  = '-'

    try:
        element_xpath_content4 = "//h3[contains(text(),'혜택 및 복지')]/parent::div" 
        list_df['content4'][i]  = driver.find_element(By.XPATH,element_xpath_content4).text.replace('혜택 및 복지\n','').replace('• ','').replace('', '')
    except:
        list_df['content4'][i]  = '-'

    try:
        element_xpath2 = "//h2[contains(text(),'마감일')]/parent::article" 
        list_df['period'][i]  = driver.find_element(By.XPATH, element_xpath2).text.replace('마감일\n','')
    except:
        list_df['period'][i]  = '-'
    
    try:
        element_xpath = "//h2[contains(text(),'기술 스택 • 툴')]/parent::article" 
        skill_list = driver.find_element(By.XPATH, element_xpath).text.replace('기술 스택 • 툴\n','').split('\n')
        list_df['skill'][i] = "::".join(skill_list)
    except:
        list_df['skill'][i]
    print(i)

3120
3121
3124
3125
3126
3127
3129
3131


In [57]:
list_df.to_excel('c://python//원티드_크롤링_IT직군대상.xlsx', index=False) 

# 크롤링 하다가 정보값이 없어 멈춰버린 경우 index 재조정시 

In [54]:
list_df.iloc[ list_index, : ].query("company == '-'").head(5) #왼쪽 최상단 index에서 부터 출발 

,job_category_id,job_category,company_id,company_name,position_name,position_id,link,top_salary,major_company,company,position,content1,content2,content3,content4,skill,period
3120,518,IT,3747,에스제이에이치스튜디오,클라이언트 개발자,144972,https://www.wanted.co.kr/wd/144972,Y,NaN,-,-,-,-,-,-,-,-
3121,518,IT,3747,에스제이에이치스튜디오,서버 개발자,163538,https://www.wanted.co.kr/wd/163538,Y,NaN,-,-,-,-,-,-,-,-
3124,518,IT,886,넥슨코리아(NEXON),[인텔리전스랩스] 미디어개발팀 웹개발자 (Node.js),120650,https://www.wanted.co.kr/wd/120650,Y,Y,-,-,-,-,-,-,-,-
3125,518,IT,43837,유니컴즈,Software Developer,165271,https://www.wanted.co.kr/wd/165271,Y,NaN,-,-,-,-,-,-,-,-
3126,518,IT,7608,골프존카운티,안드로이드 개발자,189418,https://www.wanted.co.kr/wd/189418,Y,NaN,-,-,-,-,-,-,-,-


In [55]:
list_index[np.where( list_index == 3120)[0][0]:]  #1.index 확인 후 2. 왼쪽 코드를 위 for문에 대입

array([3120, 3121, 3124, 3125, 3126, 3127, 3129, 3131], dtype=int64)

In [46]:
#원티드 자체에서 에러난 경우 결측 처리
'''
list_df['company'][2269] ='결측'
list_df['company'][2597] ='결측'
list_df['company'][3119] ='결측'
'''

"\nlist_df['company'][2269] ='결측'\nlist_df['company'][2597] ='결측'\n"

In [53]:
list_df.to_excel('c://python//원티드_241203_개발자리스트_임직원1000명이상.xlsx', index=False) 

In [52]:
list_df

,job_category_id,job_category,company_id,company_name,position_name,position_id,link
0,518,IT,886,넥슨코리아(NEXON),[인텔리전스랩스] SDK API팀 백엔드 개발자 (Java),253531,https://www.wanted.co.kr/wd/253531
0,518,IT,27352,에스케이매직(SK매직),D2C 웹사이트 구축 프로젝트 기획/운영 담당 (8년이상),253041,https://www.wanted.co.kr/wd/253041
0,518,IT,7810,한화생명보험,[COE부문] Backend Developer(Senior/PM),253582,https://www.wanted.co.kr/wd/253582
0,518,IT,521,크래프톤(Krafton),Unity Platform SDK Developer (3년 이상),253162,https://www.wanted.co.kr/wd/253162
0,518,IT,27352,에스케이매직(SK매직),LLM 솔루션 R&D(도메인 특화) (석사 이상),253192,https://www.wanted.co.kr/wd/253192
...,...,...,...,...,...,...,...
0,518,IT,21933,현대캐피탈,iOS 개발자,225949,https://www.wanted.co.kr/wd/225949
0,518,IT,521,크래프톤(Krafton),[OVERDARE] Back-end Engineer - Blockchain,203277,https://www.wanted.co.kr/wd/203277
0,518,IT,521,크래프톤(Krafton),[BATTLEGROUNDS] Anti-Cheat 플랫폼 엔지니어 (3년 이상),28199,https://www.wanted.co.kr/wd/28199
0,518,IT,886,넥슨코리아(NEXON),[인텔리전스랩스] 미디어개발팀 웹개발자 (Node.js),120650,https://www.wanted.co.kr/wd/120650


In [41]:
# 포지션 수정 
list_df['position_name2'] = list_df['position_name'].str.replace(r'\s*\([^)]*\)', '', regex=True).str.replace(r'\s*\[[^\]]*\]', '', regex=True).str.strip() 

In [39]:
list_df[['company1', 'street1', 'career1']] = list_df['company'].str.split('∙', expand=True)